In [ ]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path

from recbole.quick_start import run_recbole
# from recbole.quick_start import objective_function
# from recbole.quick_start import run_recbole
from recbole.config import Config
from recbole.data import data_preparation
from recbole.trainer import HyperTuning
from recbole.utils import (
    get_model,
    get_trainer,
)

import utils
from real_temporal import TimeCutoffDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. Define necessary paths

In [ ]:
path_logs = Path("logs")
path_configs = Path("configs")

path_logs.mkdir(parents=True, exist_ok=True)
path_configs.mkdir(parents=True, exist_ok=True)

In [3]:
config_dict = {}

# 1. Define model

In [2]:
config_dict = {
    **config_dict,

    'model': "NPE",
    
    'embedding_size': 64,
    'hidden_size': 128,
    'num_layers': 1,
    'dropout_prob': 0.3,
    'loss_type': "CE",
}

# 2. Define data

In [3]:
config_dict = {
    **config_dict,

    'dataset': 'ml-100k',

    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'load_col': {"inter": ['user_id', 'item_id', 'timestamp']},
}

# 3. Train

## 3.1. Define training and evaluating configuration

In [ ]:
config_dict = {
    **config_dict,

    'device': 'mps',
    
    # TODO: HoangLe [Jun-17]: Complete this
}

## 3.2. Start training

In [5]:
config_dict = {
    # 'eval_args': {
    #     "order": "TO",
    #     "split": {"CO": '886349689'},
    #     "group_by": 'user_id'
    # },
    'train_neg_sample_args': None
}

run_recbole(model=model, dataset=data, config_dict=config_dict)

13 Jun 20:42    INFO  ['/Users/macos/miniforge3/envs/py/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/Users/macos/Library/Jupyter/runtime/kernel-v2-47538r08pKQI58BLn.json']
13 Jun 20:42    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /Users/macos/miniforge3/envs/py/lib/python3.10/site-packages/recbole/config/../dataset_example/ml-100k
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'T

{'best_valid_score': 0.0185,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.0689),
              ('mrr@10', 0.0185),
              ('ndcg@10', 0.03),
              ('hit@10', 0.0689),
              ('precision@10', 0.0069)]),
 'test_result': OrderedDict([('recall@10', 0.0509),
              ('mrr@10', 0.0127),
              ('ndcg@10', 0.0214),
              ('hit@10', 0.0509),
              ('precision@10', 0.0051)])}

# 4. Tune hyper params

## 4.1. Define hyper params and object function

In [ ]:
hyper_params = {
    'loguniform': {
        'learning_rate': [-8, 0]
    },
    'choice': {
        'embedding_size': [64,96,128],
        'mlp_hidden_size': ['[64,64,64]','[128,128]']
    }
}

In [ ]:
def objective_function(config_dict=None, config_file_list=None):
    # TODO: HoangLe [Jun-17]: Complete this
    
    config_dict = {
        "use_gpu": True,
        "eval_args": {
            "order": "TO",
            "split": {"CO": "886349689"},
            "group_by": "user_id",
        },
        "train_neg_sample_args": None,
    }

    config = Config(
        config_dict=config_dict,
        config_file_list=config_file_list,
    )
    

    # Define data related things
    dataset = TimeCutoffDataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # Define model
    model_name = config['model']
    model = get_model(model_name)(config, train_data._dataset).to(config['device'])

    # Define trainer
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

    # Start training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, verbose=False)

    # Start evaluating
    test_result = trainer.evaluate(test_data)

    return {
        'model': model_name,
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result
    }

## 4.2. Start tuning

In [2]:
hp = HyperTuning(
    objective_function=objective_function,
    algo='bayes',
    early_stop=10,
    max_evals=100,
    params_dict=hyper_params,
    fixed_config_file_list=['test.yml']
)


hp.run()


running parameters:                                    
{'embedding_size': 64, 'learning_rate': 0.0010749028422793762, 'mlp_hidden_size': '[64,64,64]'}
current best valid score: 0.0440                       
current best valid result:                             
OrderedDict([('recall@10', 0.1175), ('mrr@10', 0.044), ('ndcg@10', 0.0611), ('hit@10', 0.1175), ('precision@10', 0.0117)])
current test result:                                   
OrderedDict([('recall@10', 0.0394), ('mrr@10', 0.0122), ('ndcg@10', 0.0182), ('hit@10', 0.0394), ('precision@10', 0.0039)])
running parameters:                                                  
{'embedding_size': 128, 'learning_rate': 0.004262748642670945, 'mlp_hidden_size': '[64,64,64]'}
running parameters:                                                  
{'embedding_size': 96, 'learning_rate': 0.10990572676884496, 'mlp_hidden_size': '[128,128]'}
running parameters:                                                  
{'embedding_size': 96, 'learning_r

## 4.3. Export tunning result

In [ ]:
# TODO: HoangLe [Jun-17]: Complete this

# export result to the file
path_log_tune = path_logs / f"{}"
# hp.export_result(output_file='hyper_example.result')
# NOTE: HoangLe [Jun-17]: May include datetime, model name, data name
# print best parameters
# print('best params: ', hp.best_params)

# print best result
# print('best result: ')
# print(hp.params2result[hp.params2str(hp.best_params)])

In [1]:
from real_temporal import run_recbole_with_TimeCutoff, TimeCutoffDataset

from recbole.trainer import HyperTuning
# from recbole.quick_start import objective_function
from recbole.config import Config
# from recbole.quick_start import run_recbole
from recbole.data import (
    create_dataset,
     data_preparation,
)

from recbole.utils import (
    get_model,
    get_trainer,
    init_seed,
)

def objective_function(config_dict=None, config_file_list=None):
    model_name = 'NPE'
    dataset_name = 'ml-100k'

    config = Config(
        config_dict=config_dict,
        config_file_list=config_file_list,
        model=model_name,
        dataset=dataset_name
    )
    # init_seed(config['seed'])
    dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)
    model_name = config['model']
    model = get_model(model_name)(config, train_data._dataset).to(config['device'])
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, verbose=False)
    test_result = trainer.evaluate(test_data)

    return {
        'model': model_name,
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result
    }

hp = HyperTuning(
    objective_function=objective_function,
    algo='bayes',
    early_stop=10,
    max_evals=100,
    params_file='model.hyper',
    fixed_config_file_list=['test.yml']
)

# run
hp.run()
# export result to the file
# hp.export_result(output_file='hyper_example.result')
# # print best parameters
# print('best params: ', hp.best_params)
# # print best result
# print('best result: ')
# print(hp.params2result[hp.params2str(hp.best_params)])

running parameters:                                    
{'embedding_size': 128, 'learning_rate': 0.012213481366920161, 'mlp_hidden_size': '[128,128]'}
current best valid score: 0.0164                       
current best valid result:                             
OrderedDict([('recall@10', 0.0668), ('mrr@10', 0.0164), ('ndcg@10', 0.0279), ('hit@10', 0.0668), ('precision@10', 0.0067)])
current test result:                                   
OrderedDict([('recall@10', 0.0435), ('mrr@10', 0.008), ('ndcg@10', 0.0161), ('hit@10', 0.0435), ('precision@10', 0.0043)])
running parameters:                                                     
{'embedding_size': 64, 'learning_rate': 0.09821758837521936, 'mlp_hidden_size': '[128,128]'}
current best valid score: 0.0218                                        
current best valid result:                                              
OrderedDict([('recall@10', 0.0647), ('mrr@10', 0.0218), ('ndcg@10', 0.0317), ('hit@10', 0.0647), ('precision@10', 0.0065)]

KeyboardInterrupt: 

In [5]:
from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

hp = HyperTuning(objective_function=objective_function, algo='bayes', early_stop=10,
                max_evals=100, params_file='model.hyper', fixed_config_file_list=['test.yml'])

# run
hp.run()
# export result to the file
hp.export_result(output_file='hyper_example.result')
# print best parameters
print('best params: ', hp.best_params)
# print best result
print('best result: ')
print(hp.params2result[hp.params2str(hp.best_params)])

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

17 Jun 22:12    INFO  build_posterior_wrapper took 0.000621 seconds

17 Jun 22:12    INFO  TPE using 0 trials



running parameters:                                    
{'embedding_size': 64, 'learning_rate': 0.0005927902936991883, 'mlp_hidden_size': '[128,128]'}
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

17 Jun 22:12    INFO  [Training]: train_batch_size = [4096] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]

17 Jun 22:12    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'None', 'mode': {'valid': 'full', 'test': 'full'}}]

17 Jun 22:12    INFO  Loading model structure and parameters from saved/ItemKNN-Jun-17-2024_22-12-52.pth



current best valid score: 0.0463                       
current best valid result:                             
OrderedDict([('recall@10', 0.1262), ('mrr@10', 0.0463), ('ndcg@10', 0.0647), ('hit@10', 0.1262), ('precision@10', 0.0126)])
current test result:                                   
OrderedDict([('recall@10', 0.1145), ('mrr@10', 0.0473), ('ndcg@10', 0.0629), ('hit@10', 0.1145), ('precision@10', 0.0115)])
  1%|          | 1/100 [00:01<02:37,  1.59s/trial, best loss: -0.0463]

...

  9%|▉         | 9/100 [00:13<02:23,  1.58s/trial, best loss: -0.0463]

17 Jun 22:13    INFO  build_posterior_wrapper took 0.000442 seconds

17 Jun 22:13    INFO  TPE using 9/9 trials with best loss -0.046300



running parameters:                                                   
{'embedding_size': 64, 'learning_rate': 0.05115291209207257, 'mlp_hidden_size': '[128,128]'}
  9%|▉         | 9/100 [00:13<02:23,  1.58s/trial, best loss: -0.0463]

17 Jun 22:13    INFO  [Training]: train_batch_size = [4096] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]

17 Jun 22:13    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'None', 'mode': {'valid': 'full', 'test': 'full'}}]

17 Jun 22:13    INFO  Loading model structure and parameters from saved/ItemKNN-Jun-17-2024_22-13-06.pth

17 Jun 22:13    INFO  Early stop triggered. Stopping iterations as condition is reach.



 10%|█         | 10/100 [00:15<02:18,  1.54s/trial, best loss: -0.0463]
best params:  {'embedding_size': 64, 'learning_rate': 0.0005927902936991883, 'mlp_hidden_size': '[128,128]'}
best result: 
{'model': 'ItemKNN', 'best_valid_score': 0.0463, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.1262), ('mrr@10', 0.0463), ('ndcg@10', 0.0647), ('hit@10', 0.1262), ('precision@10', 0.0126)]), 'test_result': OrderedDict([('recall@10', 0.1145), ('mrr@10', 0.0473), ('ndcg@10', 0.0629), ('hit@10', 0.1145), ('precision@10', 0.0115)])}


# 5. Offline evaluation

In [ ]:
EVAL_MODES = {
    "temporal": "TO",
    "leave1out": "LS"
}

In [ ]:
selected_eval_models = [EVAL_MODES[k] for k in ["temporal", "leave1out"]]
selected_eval_metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
selected_topk = 10

eval_args = {
    "order": "TO",
    "split": {
        'LS': "valid_and_test"
    },
    "group_by": None
}

config_dict = {
    'eval_args': eval_args,
    # 'train_neg_sample_args': None,
}

run_recbole(model=model, dataset=data, config_dict=config_dict)

In [ ]:
run_recbole(model='NPE', dataset='ml-100k', config_file_list=['test.yml'])